In [1]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

import torchvision
from torchvision import transforms

from tqdm import tqdm
from PIL import Image

## Loading the datasets from image folders & performing transforms 

In [2]:
train_data_path = "./dataset/train/"
val_data_path = "./dataset/val/"
test_data_path = "./dataset/test/"

transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transform)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)

## Creating the dataloaders

In [3]:
batch_size=64

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

## Creating the model

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(3 * 64 * 64, 240)
        self.fc2 = nn.Linear(240, 100)
        self.fc3 = nn.Linear(100,2)
        
    def forward(self, x):
        x = x.view(-1, 3 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

mlp = MLP()

## Creating the optimizer & loss function

In [5]:
optimizer = optim.Adam(mlp.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

## Running the training loop

In [6]:
def train(model, optimizer, loss_fn, train_loader, val_loader,
    epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, target = batch
            inputs = inputs
            output = model(inputs)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item()
        training_loss /= len(train_loader)
        model.eval()
        
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, target = batch
            inputs = inputs
            output = model(inputs)
            loss = loss_fn(output,target)
            valid_loss += loss.data.item()
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],
            target).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader)
        
        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, accuracy = {:.2f}'.format(
            epoch, training_loss, valid_loss, num_correct / num_examples))

In [7]:
train(mlp, optimizer, loss_fn, train_loader, val_loader)

Epoch: 0, Training Loss: 0.8991, Validation Loss: 0.9808, accuracy = 0.55
Epoch: 1, Training Loss: 0.5792, Validation Loss: 0.8723, accuracy = 0.60
Epoch: 2, Training Loss: 0.4506, Validation Loss: 0.7404, accuracy = 0.55
Epoch: 3, Training Loss: 0.3040, Validation Loss: 1.0787, accuracy = 0.61
Epoch: 4, Training Loss: 0.2439, Validation Loss: 0.9830, accuracy = 0.63
Epoch: 5, Training Loss: 0.1703, Validation Loss: 0.9692, accuracy = 0.59
Epoch: 6, Training Loss: 0.0860, Validation Loss: 1.1752, accuracy = 0.60
Epoch: 7, Training Loss: 0.0566, Validation Loss: 1.0742, accuracy = 0.60
Epoch: 8, Training Loss: 0.0360, Validation Loss: 1.3309, accuracy = 0.62
Epoch: 9, Training Loss: 0.0272, Validation Loss: 1.2562, accuracy = 0.59
Epoch: 10, Training Loss: 0.0239, Validation Loss: 1.3383, accuracy = 0.58
Epoch: 11, Training Loss: 0.0280, Validation Loss: 1.2499, accuracy = 0.60
Epoch: 12, Training Loss: 0.0136, Validation Loss: 1.5592, accuracy = 0.60
Epoch: 13, Training Loss: 0.0109, V

## Making predictions

In [8]:
FILENAME = 'dataset/test/cat/cat_6.jpg'
labels = ['cat','dog']
img = Image.open(FILENAME)
img = transform(img)
img = img.unsqueeze(0) #adds a new dimension at the front of the tensor
prediction = mlp(img)
prediction = prediction.argmax()
print(labels[prediction])

cat


## Model saving

In [13]:
save_path = 'pretrained_models/mlp.pt'
torch.save(mlp.state_dict(), save_path)

## Loading model

In [15]:
mlp = MLP()
mlp_state_dict = torch.load(save_path)
mlp.load_state_dict(mlp_state_dict)

<All keys matched successfully>